In [1]:
#https://www.kaggle.com/vatsalmavani/music-recommendation-system-using-spotify-dataset

In [9]:
# IMPORTANTE para saber como explicar el proyecto

#https://www.youtube.com/watch?v=TTL3iBziKqY --> para darle mas fondo al proyecto

In [12]:
#WORTH MIRARSE!!!

#https://medium.com/@david.de.hernandez/modeling-data-for-a-spotify-recommender-system-3056997a0fc5
#https://levelup.gitconnected.com/music-recommender-system-part-6-4278bf89411c

# Imports

In [5]:
import os
import numpy as np
import pandas as pd

import seaborn as sns
import plotly.express as px 
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cdist

import warnings
warnings.filterwarnings("ignore")

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from collections import defaultdict


from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cdist
import difflib

---
# Read Data

In [15]:
data = pd.read_csv('./data/JOINT.csv')

data.drop(columns = ['time_signature'], inplace = True)

# Drop rows containing null values as they aren't a significant number
data.dropna(inplace = True)
data.drop_duplicates(['id'], inplace = True)

#change release_date to datetime
data['release_year'] = data['release_date'].apply(lambda x: x.split('-')[0])
data.drop(columns = ['release_date'], inplace = True)

#Change duration in ms to seconds and drop duration_ms column
data['duration'] = data['duration_ms'].apply(lambda x: round(x/1000))
data.drop(columns = ['duration_ms'], inplace = True)



#######################################################################
song_cluster_pipeline = Pipeline([('scaler', StandardScaler()), 
                                  ('kmeans', KMeans(n_clusters=4, 
                                   verbose=False, n_jobs=4))
                                 ], verbose=False)

X = data.select_dtypes(np.number)
number_cols = list(X.columns)
song_cluster_pipeline.fit(X)
song_cluster_labels = song_cluster_pipeline.predict(X)
data['cluster_label'] = song_cluster_labels

#######################################################################
SPOTIFY_CLIENT_ID = 'daf844afde2c4b25840254841b7952f9'
SPOTIFY_CLIENT_SECRET = 'd82684f896274fae917f37a1ff88b2fb'

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=SPOTIFY_CLIENT_ID, client_secret=SPOTIFY_CLIENT_SECRET))

def find_song(title, artist):
    song_data = defaultdict()
    #results = sp.search(q= 'track: {}'.format(title), limit=1)
    results = sp.search(q = 'track: {} artist: {}'.format(title, artist), limit=1, offset=0, type='track')
    if results['tracks']['items'] == []:
        return None

    results = results['tracks']['items'][0]
    track_id = results['id']
    audio_features = sp.audio_features(track_id)[0]

    song_data['title'] = [title]
    song_data['artist'] = [artist]
    song_data['duration_ms'] = [results['duration_ms']]
    song_data['popularity'] = [results['popularity']]

    for key, value in audio_features.items():
        #key = feature and value = value of feature
        song_data[key] = value
    
    df = pd.DataFrame(song_data)
    
    #CAMBIOS DE MI FEATURE ENGINEERING    
    df.drop(columns = ['time_signature'], inplace = True)

    # Drop rows containing null values as they aren't a significant number
    df.dropna(inplace = True)
    df.drop_duplicates(['id'], inplace = True)

    #Change duration in ms to seconds and drop duration_ms column
    df['duration'] = df['duration_ms'].apply(lambda x: round(x/1000))
    df.drop(columns = ['duration_ms'], inplace = True)
        
    return df



#######################################################################

#Song_list must be a list of tuples --> [('title', 'artist')]
def get_mean_vector(song_list):
    
    song_vectors = []
    
    for song in song_list:
        song_data = find_song(song[0], song[1])
        if song_data is None:
            print('Warning: {} from {} does not exist in Spotify or in database'.format(song['title'], song['artist']))
            continue
        song_vector = song_data[number_cols].values
        song_vectors.append(song_vector)  
    
    song_matrix = np.array(list(song_vectors))
    return np.mean(song_matrix, axis=0)

#######################################################################

def recommend_songs(song_list, tracks_data=data, n_songs=10):
    
    metadata_cols = ['title', 'all_artists']
    
    song_center = get_mean_vector(song_list)
    scaler = song_cluster_pipeline.steps[0][1] #Instantiates StandardScaler()
    scaled_data = scaler.transform(tracks_data[number_cols])
    scaled_song_center = scaler.transform(song_center.reshape(1, -1))
    distances = cdist(scaled_song_center, scaled_data, 'cosine')
    index = list(np.argsort(distances)[:, :n_songs][0])
    
    rec_songs = tracks_data.iloc[index]
    return rec_songs[['title', 'all_artists']]

#######################################################################


input_song = input('Enter song name:')
input_artist = input('Enter artist name:')
list_of_songs = []
list_of_songs.append((input_song, input_artist))

recommend_songs(song_list = list_of_songs)

Enter song name: Hello
Enter artist name: Adele


,title,all_artists
3819,Hello,Adele
28246,Poplar St,Glass Animals
14740,I Swear,All-4-One
3519,Love Me Like You Do,Ellie Goulding
4443,Eternal Flame,The Bangles
30000,Goner,Twenty One Pilots
27410,Keep Your Head Up,Ben Howard
25307,Nights,Frank Ocean
27150,You & Me - Flume Remix,Flume
25034,The Weekend,SZA


In [14]:
SPOTIFY_CLIENT_ID = 'daf844afde2c4b25840254841b7952f9'
SPOTIFY_CLIENT_SECRET = 'd82684f896274fae917f37a1ff88b2fb'

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=SPOTIFY_CLIENT_ID, client_secret=SPOTIFY_CLIENT_SECRET))

def find_song(lista):
    inp = pd.DataFrame()
    
    for (title, artist) in lista:
        song_data = defaultdict()
        #results = sp.search(q= 'track: {}'.format(title), limit=1)
        results = sp.search(q = 'track: {} artist: {}'.format(title, artist), limit=1, offset=0, type='track')
        if results['tracks']['items'] == []:
            return None

        results = results['tracks']['items'][0]
        track_id = results['id']
        audio_features = sp.audio_features(track_id)[0]

        song_data['title'] = [title]
        song_data['artist'] = [artist]
        song_data['duration_ms'] = [results['duration_ms']]
        song_data['popularity'] = [results['popularity']]

        for key, value in audio_features.items():
        #key = feature and value = value of feature
            song_data[key] = value
    
        df = pd.DataFrame(song_data)
    
        #CAMBIOS DE MI FEATURE ENGINEERING    
        df.drop(columns = ['time_signature'], inplace = True)

        # Drop rows containing null values as they aren't a significant number
        df.dropna(inplace = True)
        df.drop_duplicates(['id'], inplace = True)

        #Change duration in ms to seconds and drop duration_ms column
        df['duration'] = df['duration_ms'].apply(lambda x: round(x/1000))
        df.drop(columns = ['duration_ms'], inplace = True)
        inp = pd.concat([inp, df])
    return inp

#input_song = input('Enter song name:')
#input_artist = input('Enter artist name:')
list_of_songs = []
while len(list_of_songs)<2:
    input_song = input('Enter song name:')
    input_artist = input('Enter artist name:')
    list_of_songs.append((input_song, input_artist))

input_df = find_song(list_of_songs)
input_df.head()

Enter song name: Hello 
Enter artist name: Adele
Enter song name: Better together
Enter artist name: Jack Johnson


,title,artist,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration
0,Hello,Adele,76,0.578,0.430,5,-6.134,0,0.0305,0.330,0.00000,0.0854,0.288,78.991,audio_features,62PaSfnXSMyLshYJrlTuL3,spotify:track:62PaSfnXSMyLshYJrlTuL3,https://api.spotify.com/v1/tracks/62PaSfnXSMyL...,https://api.spotify.com/v1/audio-analysis/62Pa...,296
0,Better together,Jack Johnson,71,0.868,0.347,5,-9.529,1,0.0536,0.308,0.00002,0.1100,0.659,110.059,audio_features,4VywXu6umkIQ2OS0m1I79y,spotify:track:4VywXu6umkIQ2OS0m1I79y,https://api.spotify.com/v1/tracks/4VywXu6umkIQ...,https://api.spotify.com/v1/audio-analysis/4Vyw...,208
